In [1]:
#!pip install --user -r requirements.txt

In [2]:
import sys
import os
# Set up TWINT config
import twint
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio

# stopword
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw_nltk = stopwords.words('english')


nest_asyncio.apply()
c = twint.Config()
c.Search = "BTC"
c.Lang = "en"
# c.Username = "noneprivacy"
# c.Custom["tweet"] = ["id"]
# c.Custom["user"] = ["bio"]

c.Store_pandas = True
# c.Store_csv = True
# c.Output = "test.csv"
c.Limit = 100
c.Pandas = True
twint.run.Search(c)

[nltk_data] Downloading package stopwords to /Users/swu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1491300578236178434 2022-02-08 22:38:59 -0800 <STLdocks> $IOTX and $ACH are still doing very well against the fall back in btc. The things that still give me hope while I wait on $jasmy $xyo and the never up $spell to come back up lmao!! Alt season is going to be great!
1491300577258909696 2022-02-08 22:38:59 -0800 <Bentley11863142> SmartDeFi™ by FEG – Build Your Next Token the SmartDeFi™ Way  https://t.co/evCjdhxNBw  @GGGSmartDefi @BTC_Archive @CoinMarketCap @davidgokhshtein @Next100XGEMS @IvanOnTech @cryptomanran @AltcoinDailyio @TheCryptoLark @BCBacker @Bitboy_Crypto @MMCrypto @cryptojack @ChinaPumpWXC
1491300569423749120 2022-02-08 22:38:57 -0800 <BitCoinAlertBot> #BTC has risen by 13.76% in the last week. The price is currently $43701.59.
1491300565133172736 2022-02-08 22:38:56 -0800 <umayahsepti> Trending on #LunarCrush:  "Google Cloud launches agentless cryptojacking malware scanner | ZDNet" via @ZDNet  Top coin mentions $btc   https://t.co/PC5o5TUJTj
1491300548230131713 2022-02

1491300166506524675 2022-02-08 22:37:21 -0800 <iamJasonSharp> Thoughts on BTC.. Since we now have futures contracts on BTC, like gold, people no longer need the underlying asset to trade it, they use paper contracts.   Now the two of three biggest powers have an agreement 🤝, in my opinion, to manipulate the price.  https://t.co/fXB6BWGkJC
1491300165025943552 2022-02-08 22:37:21 -0800 <nono01022> @INVESTMENTSHULK @captainweb  Contexte : ces gens sont suspecté d’avoir participé au Hack de Bitfinex en 2016 ( 160k BTC = 3,6 MIYIARS DE $) .  Un pur produit de l’intelligence humaine.  Ils sont été arrêtés par la CIA mardi.
1491300157497163776 2022-02-08 22:37:19 -0800 <bittu_crypto> the strongest bullish signal flashed for #Bitcoin. Not trend reverse but price bounce is anticipated!  @Tradeosiann  is a super #crypto Trader  I’ve been following his tweets and tips seriously &amp; I’ve been doing great #Metaverse #BTC #DOGE #NFT🤞
1491300151415435267 2022-02-08 22:37:17 -0800 <popstarkiller> st

1491299834657390592 2022-02-08 22:36:02 -0800 <bittu_crypto> the strongest bullish signal flashed for #Bitcoin. Not trend reverse but price bounce is anticipated!  @Tradeosiann  is a super #crypto Trader  I’ve been following his tweets and tips seriously &amp; I’ve been doing great #Metaverse #BTC #DOGE #NFT÷×
1491299832996446209 2022-02-08 22:36:01 -0800 <shibucr4> Trending on #LunarCrush:  "Ukrainians are using bitcoin to crowdfund Russian war, says blockchain analyst" via @MarketWatch  Top coin mentions $btc   https://t.co/t4Aes12UNs
1491299831478112256 2022-02-08 22:36:01 -0800 <saamon2500> BTCはたかしが触り始めたら触ることにします。
1491299826159734785 2022-02-08 22:36:00 -0800 <EmikoSevern> ビットコインやリップル、イーサリアム等の仮想通貨を取引してる方、興味がある方、日本初海外取引所bitarzに無料登録で1万円分のbtcが貰えます。この資金でレバレッジ888倍のfx取引をして資金を一気に増やしませんか？ゼロカットでリスクなし登録はメアドだけで可能  https://t.co/HQqi2JL7QV
1491299823164993537 2022-02-08 22:35:59 -0800 <vijay5972> @WazirXCares @ajmerapriyansh2 @NischalShetty @shardeum Add #BabyDoge also..Just for Fun like Shib n

In [4]:
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
df = twint_to_pandas(["date", "username","tweet", "hashtags", "nreplies", "nretweets","nlikes"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       105 non-null    object
 1   username   105 non-null    object
 2   tweet      105 non-null    object
 3   hashtags   105 non-null    object
 4   nreplies   105 non-null    int64 
 5   nretweets  105 non-null    int64 
 6   nlikes     105 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 5.9+ KB


In [5]:
df.head(5)

,date,username,tweet,hashtags,nreplies,nretweets,nlikes
0,2022-02-08 22:38:59,STLdocks,$IOTX and $ACH are still doing very well again...,[],0,0,0
1,2022-02-08 22:38:59,Bentley11863142,SmartDeFi™ by FEG – Build Your Next Token the ...,[],0,0,0
2,2022-02-08 22:38:57,BitCoinAlertBot,#BTC has risen by 13.76% in the last week. The...,[btc],0,0,0
3,2022-02-08 22:38:56,umayahsepti,"Trending on #LunarCrush: ""Google Cloud launch...",[lunarcrush],0,0,0
4,2022-02-08 22:38:52,bava23,@safalniveshak #btc works 24/7,[btc],0,0,0


## Sentiment analysis

### 1. Preprocessing

In [9]:
def clean_anonymization(tweet):
    res=[]
    # Lowercase
    tweet = tweet.lower()
    # Remove single letter words
    # tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    # Remove stopword
    tweet = ' '.join([word for word in tweet.split() if word.lower() not in sw_nltk])
    for i in tweet.split():
        # remove the word after@
        if i.startswith("@") or i.startswith("#"):
            continue
        else:
            res.append(i)
    return ' '.join(res)

In [10]:
df['tweet']=df['tweet'].apply(lambda x:clean_anonymization(x))

<ipython-input-10-79f1e2b532d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet']=df['tweet'].apply(lambda x:clean_anonymization(x))


In [11]:
df.head()

,date,username,tweet,hashtags,nreplies,nretweets,nlikes
0,2022-02-08 22:38:59,STLdocks,$iotx $ach still well fall back btc. things st...,[],0,0,0
1,2022-02-08 22:38:59,Bentley11863142,smartdefi™ feg – build next token smartdefi™ w...,[],0,0,0
2,2022-02-08 22:38:57,BitCoinAlertBot,risen 13.76% last week. price currently $43701...,[btc],0,0,0
3,2022-02-08 22:38:56,umayahsepti,"trending ""google cloud launches agentless cryp...",[lunarcrush],0,0,0
4,2022-02-08 22:38:52,bava23,works 24/7,[btc],0,0,0


### 2. Stacked-LSTM 

In [ ]:
# Example of one output for each input time step
from keras.models import Sequential
from keras.layers import LSTM
from numpy import array
# define model where LSTM is also output layer
model = Sequential()
model.add(LSTM(1, return_sequences=True, input_shape=(3,1)))
model.compile(optimizer='adam', loss='mse')
# input time steps
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

In [13]:
from textblob import TextBlob
from IPython.display import Markdown, display

In [14]:
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [15]:
for tweet in df['tweet']:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("Neutral", color="grey")
        print("")

@Israelm16432 @Tarjas5 @Cryptobullza @BTC_Archive So this happened to me for 3-weeks and I tried so many means but all failed, luckily for me I got in contact with this Hacker pro.craxk on Instagram, he was the one who helped me, he should help you with yours.
Sentiment(polarity=0.1111111111111111, subjectivity=0.5444444444444444)


<span style='color:green'>Positive</span>

@Barbarasllv6 Boa noite 🔥 chegou com fogo na btc né
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>


I had a dream that #BTC reached $ 79,000🤔
Sentiment(polarity=0.0, subjectivity=0.0)


...

<span style='color:green'>Positive</span>

Tryna fly bae overseas but we up &amp; down like BTC
Sentiment(polarity=0.32222222222222224, subjectivity=0.5944444444444444)


<span style='color:green'>Positive</span>

@saylor #BTC Bossssss 🔥💀🌹🙌💎
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>

In [16]:
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
# Function to get sentiment 
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative
# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
def tweets_sentiment(search, limit=1):
    c.Search = search
    # Custom output format
    c.Format = "Username: {username} |  Tweet: {tweet}"
    c.Limit = limit
    c.Pandas = True
    with HiddenPrints():
        print(twint.run.Search(c))
    
    # Transform tweets to pandas DF
    df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
    
    # Transform Pandas DF to Optimus/Spark DF
    df = op.create.data_frame(pdf= df_pd)
    
    # Clean tweets
    clean_tweets = df.cols.remove_accents("tweet") \
                 .cols.remove_special_chars("tweet")
    
    # Add sentiment to final DF
    return clean_tweets.withColumn("sentiment",    sentiment(clean_tweets['tweet']))

## Add sentiments to dataframe